In [1]:
from numpy import sqrt, pi, exp, arcsin, linspace, loadtxt, zeros, diag
from pandas import DataFrame
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

In [2]:
def plot(x0, D_obs, *args):
    fig  = plt.figure()
    axes = fig.add_subplot(1,1,1)
    axes.plot(x0, D_obs, 'o', linewidth=3, color='#90CAF9', label='D (Observed)')
    axes.plot(x0, oblate(x0, *args), linewidth=2, color='#BA68C8', label='D (Oblate Elipsoid Model)')
    axes.spines['top'].set_visible(False)
    axes.spines['right'].set_visible(False)
    axes.xaxis.set_ticks_position('bottom')
    axes.yaxis.set_ticks_position('left')
    axes.set_xlabel('Concentration')
    axes.set_ylabel('D')
    axes.legend()
    axes.grid(alpha=.5)
    plt.show()

def show_params(D1, D2, K, d):
    table_data = {
        'D1'   : D1,
        'D2'   : D2,
        'D1/D2': D1/D2,
        'K'    : K,
        'd'    : d
    }
    props_table = DataFrame(
        table_data,
        index=[0],
        columns=['D1', 'D2', 'D1/D2', 'K', 'd']
    )
    return props_table.style

# def oblate(x0, D1, D2, K, d):
#     k = 1.38e-23
#     T = 298
#     L = 0.34
#     eta = 0.00125
#     x1 = (1+2*K*x0-(1+4*K*x0)**.5)/(2*(K**2)*x0)
    
#     SUMM = zeros(len(x1))
#     for i in range(3, 100):
#         av = sqrt( 1-(d/(L*i))**2 )  # additional variable
#         SUMM += i**(1/3) * (K*x1)**(i-1) * ( arcsin(av) / av )
    
#     D = (x1/x0) * ( D1 + 2*K*x1*D2 + (k*T/(3*pi*eta*L**(2/3)*d**(1/3))) * SUMM)
#     return D

def oblate(x0, D1, D2, K, d):
    L = 0.34e-9
    x1 = (1+2*K*x0-(1+4*K*x0)**.5)/(2*(K**2)*x0)
    
    SUMM = zeros(len(x1))
    for i in range(3, 100):
        av = sqrt( 1-(d/(L*i))**2 )  # additional variable
        SUMM += i**(1/3) * (K*x1)**(i-1) * ( arcsin(av) / av )
    
    D = (x1/x0) * ( D1 + 2*K*x1*D2 + (7.166e-13/(d**(1/3))) * SUMM)
    return D

In [3]:
aik_data = loadtxt('data_files\\aik.dat')
x0 = aik_data[:,0] 
D_obs = aik_data[:,1] * 1e10

print(x0)
print(D_obs)

[ 5.   4.5  4.   3.8  3.2  2.   1.   0.8  0.4  0.2]
[ 0.842  0.744  0.848  0.902  0.932  1.024  1.188  1.338  1.507  1.671]


In [4]:
(D1, D2, K, d), cov = curve_fit(
        oblate, x0, D_obs,
        jac='3-point',
        p0=[5, 5, .4,1.5e-9],
        loss='soft_l1',
        bounds=([1,0,.05,3e-10], [10,10,4,1.5e-8])
    )

plot(x0, D_obs, D1, D2, K, d)
show_params(D1, D2, K, d)

C:\Soft\Anaconda3\lib\site-packages\ipykernel\__main__.py:52: RuntimeWarning: invalid value encountered in sqrt


ValueError: Residuals are not finite in the initial point.

In [7]:
caf_data = loadtxt('data_files\\caf.dat')
x0 = caf_data[:,0]
D_obs = caf_data[:,1]*1e10

(D1, D2, K, d), cov = curve_fit(
        oblate, x0, D_obs,
        p0=[5, 5, .012,1.5e-9],
        bounds=([1,0,.008,.3e-9], [10,10,.13,3e-9])
    )

plot(x0, D_obs, D1, D2, K, d)
show_params(D1, D2, K, d)

C:\Soft\Anaconda3\lib\site-packages\ipykernel\__main__.py:52: RuntimeWarning: invalid value encountered in sqrt


ValueError: Residuals are not finite in the initial point.